<a href="https://colab.research.google.com/github/ferugit/speaker-recognition-exploration/blob/testing/model's_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spkrec-ECAPA-VoxCeleb model EER


In [ ]:
import torch
import numpy as np
from torch.nn.functional import cosine_similarity
import pandas as pd
from sklearn.metrics import roc_curve, auc

In [ ]:
def verify_files(embeddings, embeddings_1):
    cosine_similarity = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
    output = cosine_similarity(embeddings, embeddings_1)
    return output[0]

In [ ]:
df = pd.read_csv('path_to_the_embeddings_file', header = 0, sep = '\t')

D = df['embeddings']
names = df['name']

# just keep the speaker's name
remove_start = 32
remove_end = 9
embeddingsList= D.str[6:]
modified_names = names.str[remove_start:-remove_end]


D = embeddingsList.tolist()
A = modified_names.tolist()

speaker_files = list(zip(A, D))
pairs = []
for Speaker, Embedd in speaker_files:
    tensor_list = eval(Embedd)
    Embedd = torch.tensor(tensor_list)
    for Speaker2,embedd2 in speaker_files:
        pairs.append([Speaker,Speaker2,Embedd,embedd2])

In [ ]:
true_labels = []
scores = []
for pair in pairs:
    speaker1, speaker2, Embedd, embedd2 = pair
    print(f"{speaker1} and {speaker2}")
    tensor_list = eval(embedd2)
    embedd2 = torch.tensor(tensor_list)


    new_score = verify_files(Embedd,embedd2)
    # Label: 1 if same speaker, 0 if different speakers
    label = int(speaker1 == speaker2)
    true_labels.append(label)
    scores.append(new_score)

In [ ]:
# Compute the False Positive Rate (FPR) and True Positive Rate (TPR)
fpr, tpr, thresholds = roc_curve(new_true_labels, new_scores, pos_label=1)

# Find the EER threshold
eer_threshold = thresholds[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Calculate the EER
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Print the EER
print(f"Equal Error Rate  = {eer}, and the treshold = {eer_threshold}")
print(f"EER: {eer:.2f}")


In [ ]:
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Guess')
#plt.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()
plt.savefig('roc_curve.png')

# Wavlm-base model EER

In [ ]:
def similarity(embed1,embed2):
    normalized_embeddings1 = torch.nn.functional.normalize(embed1, dim=-1)
    normalized_embeddings2 = torch.nn.functional.normalize(embed2, dim=-1)

    # # # Calculate cosine similarity
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarity = cosine_sim(normalized_embeddings1, normalized_embeddings2).item()
    return similarity

In [ ]:
df = pd.read_csv('path_to_the_embeddings_file', header = 0, sep = '\t')

names = df['name']
df['embeddings'] = df['embeddings'].str.replace('\s+', ',', regex=True)

# just keeping the speaker's name from the audio file
remove_start = 30
remove_end = 9
modified_names = names.str[remove_start:-remove_end]


A = modified_names.tolist()
D = df['embeddings'].tolist()

speaker_files = list(zip(A, D))
pairs = []
for Speaker, Embedd in speaker_files:
    tensor_list = ast.literal_eval(Embedd)
    Embedd = torch.tensor(tensor_list)
    for Speaker2,embedd2 in speaker_files:
        pairs.append([Speaker,Speaker2,Embedd,embedd2])


In [ ]:
true_labels = []
scores = []
for pair in pairs:
    speaker1, speaker2, Embedd, embedd2 = pair
    print(f"{speaker1} and {speaker2}")
    tensor_list = ast.literal_eval(embedd2)
    embedd2 = torch.tensor(tensor_list)


    new_score = similarity(Embedd,embedd2)
    # Label: 1 if same speaker, 0 if different speakers
    label = int(speaker1 == speaker2)
    true_labels.append(label)
    scores.append(new_score)

In [ ]:
# Compute the False Positive Rate (FPR) and True Positive Rate (TPR)
fpr, tpr, thresholds = roc_curve(new_true_labels, new_scores, pos_label=1)

# Find the EER threshold
eer_threshold = thresholds[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Calculate the EER
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Print the EER
print(f"Equal Error Rate  = {eer}, and the treshold = {eer_threshold}")
print(f"EER: {eer:.2f}")

roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Guess')
#plt.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()
plt.savefig('roc_curve.png')